# Homework: 使用 AWQ 量化 Facebook OPT-6.7B 模型

In [1]:
from awq import AutoAWQForCausalLM
from transformers import AutoTokenizer

model_name_or_path = "facebook/opt-6.7b"
quant_model_dir = "models/opt-6.7b-awq"

quant_config = {
    "zero_point": True,
    "q_group_size": 64,
    "w_bit": 4,
    "version": "GEMM"
}

In [2]:
# 加载模型
model = AutoAWQForCausalLM.from_pretrained(model_name_or_path, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)

/root/miniconda3/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
# 量化模型
model.quantize(tokenizer, quant_config=quant_config)

Repo card metadata block was not found. Setting CardData to empty.
AWQ: 100%|██████████| 32/32 [14:20<00:00, 26.90s/it]


In [4]:
# 保存模型权重
model.save_quantized(quant_model_dir)
# 保存分词器
tokenizer.save_pretrained(quant_model_dir)  

('models/opt-6.7b-awq/tokenizer_config.json',
 'models/opt-6.7b-awq/special_tokens_map.json',
 'models/opt-6.7b-awq/vocab.json',
 'models/opt-6.7b-awq/merges.txt',
 'models/opt-6.7b-awq/added_tokens.json',
 'models/opt-6.7b-awq/tokenizer.json')

In [5]:
model.eval()

OptAWQForCausalLM(
  (model): OPTForCausalLM(
    (model): OPTModel(
      (decoder): OPTDecoder(
        (embed_tokens): Embedding(50272, 4096, padding_idx=1)
        (embed_positions): OPTLearnedPositionalEmbedding(2050, 4096)
        (final_layer_norm): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (layers): ModuleList(
          (0-31): 32 x OPTDecoderLayer(
            (self_attn): OPTAttention(
              (k_proj): WQLinear_GEMM(in_features=4096, out_features=4096, bias=True, w_bit=4, group_size=64)
              (v_proj): WQLinear_GEMM(in_features=4096, out_features=4096, bias=True, w_bit=4, group_size=64)
              (q_proj): WQLinear_GEMM(in_features=4096, out_features=4096, bias=True, w_bit=4, group_size=64)
              (out_proj): WQLinear_GEMM(in_features=4096, out_features=4096, bias=True, w_bit=4, group_size=64)
            )
            (activation_fn): ReLU()
            (self_attn_layer_norm): LayerNorm((4096,), eps=1e-05, elementwise_affine=Tr

## 使用 GPU 加载量化模型

In [6]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from awq import AutoAWQForCausalLM

quant_model_dir = "models/opt-6.7b-awq"
tokenizer = AutoTokenizer.from_pretrained(quant_model_dir)
model = AutoAWQForCausalLM.from_pretrained(quant_model_dir, device_map="cuda").to(0)

In [7]:
def generate_text(text):
    inputs = tokenizer(text, return_tensors="pt").to(0)

    out = model.generate(**inputs, max_new_tokens=64)
    return tokenizer.decode(out[0], skip_special_tokens=True)

In [8]:
result = generate_text("Merry Christmas! I'm glad to")
print(result)

Merry Christmas! I'm glad to have been been the most, merry Merry! It's been be be be I be be, Merry Merry!  If it's going be be Merry Happy Christmas!  Happy holidays, Merry Christmas and Merry Merry Christmas!  May it's Merry, Merry Christmas and Merry Merry Christmas!
*Merry Christmas
*


In [9]:
result = generate_text("The woman worked as a")
print(result)

The woman worked as a nurse and was living in one of those houses's houses which was in one of the men's houses.
The one of the one
 of the one of the one of the one of the one of the one of the one of the one of the one of the one of the one of the one of the one
